<a href="https://colab.research.google.com/github/ad451/Stackoverflow_tag_generator/blob/main/StackOverflow_Tag_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Importing the required modules**

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import random
from bs4 import BeautifulSoup
import time
import warnings; warnings.simplefilter('ignore')
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn import model_selection, naive_bayes, svm
import gensim
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


nltk.download('averaged_perceptron_tagger') # required for parts of speech
nltk.download('wordnet') # required for parts of speech
nltk.download('stopwords') #download the stopwords


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Web Scraping the current set of questions for testing the model**

In [ ]:


Questions=[] #array to store the questions

for pageNumber in range(1,401):
    response=requests.get("https://stackoverflow.com/questions",params={"tab":"newest","page":pageNumber,"pagesize":50})

    data=BeautifulSoup(response.text,'html.parser' ) #parsing the html text

    req_data=data.find(id="questions")

    new_data=req_data.find_all("h3", class_="s-post-summary--content-title") # the tag that contains the question info



    for element in new_data:
        link=element.a.attrs['href']

        response=requests.get(f"https://stackoverflow.com/{link}")  #fetching the content related to the question

        data_questionwise=BeautifulSoup(response.text,'html.parser')

        question_wise_title=data_questionwise.find("div",id="question-header").h1.a.string #title

        question_wise_desc=data_questionwise.find("div",class_="s-prose js-post-body") #description

        all_paragraphs=question_wise_desc.find_all("p")

        total_description_question_wise=""

        for para in all_paragraphs:
            total_description_question_wise+=para.text

        Final_content=question_wise_title+""+total_description_question_wise  #concatenating the title and description

        tag_question_wise=data_questionwise.find("ul",class_="ml0 list-ls-none js-post-tag-list-wrapper d-inline").li.text #tag

        Questions.append([Final_content,tag_question_wise])

    print(pageNumber)   #checking which page questions have been fetched yet

print(len(Questions))








     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 3.2 MB/s eta 0:00:00


**Sql code to combine the input questions and tags table**

In [ ]:
###################################### Code for inner combine ######################################
df1 = pd.read_csv('Questions.csv', encoding='ISO-8859-1')
df2 = pd.read_csv('Tags.csv', encoding='ISO-8859-1')

# combined dataframe of questiontags
df3 = df1.set_index('Id').join(df2.set_index('Id'))
df3 = df3.reset_index()
df4 = pd.DataFrame(df3.Id.value_counts())
df4 = df4.reset_index()

# df5 is the final combined dataframe of questiontags. Randomly picked datapoints from df3
store = {}
df5 = pd.DataFrame()
n_train = 20000
for i in range(n_train):
    key = 0
    while(key==0):
        a = random.randint(df4.shape[0])
        if a in store:
            continue
        else:
            key==1
            store[a] = 1
            ind = df3[df3.Id==df4.iloc[a,0]].first_valid_index()
            df5 =  pd.concat([df5,pd.DataFrame(df3.iloc[ind]).T],ignore_index=True)
            break

###################################### Code for removing angular brackets from the "Body" column ######################################

for j in range(df5.shape[0]):
    ans=" "
    curr = 0
    flag = 0
    while(curr<len(df5['Body'][j])):
        if( df5['Body'][j][curr]=='<'):
            flag = 1
            curr = curr + 1
            continue
        if( df5['Body'][j][curr]=='>'):
            flag = 0
            curr = curr + 1
            continue
        if(flag == 1):
            curr = curr + 1
        if(flag == 0):
            ans = ans + df5['Body'][j][curr]
            curr = curr + 1
    df1['Title'][j] += ans

df6 = pd.concat([df5.iloc[:,6], df5.iloc[:,8]], axis=1)

**Machine learning part (Preprocessing and training)**

---



In [3]:
train_data=pd.read_csv("questiontags_train.csv")
test_data=pd.read_csv("questiontags_test.csv")

In [4]:
#rename the columns of the train dataset

train_data.drop(train_data.columns[0],axis=1,inplace=True)


train_data.rename(columns={"Title":"Questions","Tag":"Tags"},inplace=True)


In [5]:
import random
#code to include around 10000 more questions to the train dataset from the test dataset
v={}
count=0
while count!=10000:
    idx=random.randint(0,test_data.shape[0]-1)
    if idx not in v:
      train_data=train_data.append(test_data.loc[idx],ignore_index=True)
      test_data.drop(idx,inplace=True,axis=0)
      count+=1
      v[idx]=1
    else:
      continue


In [6]:
train_data['Questions']=train_data['Questions'].str.replace('</p>',' ')
train_data['Questions']=train_data['Questions'].str.replace('\n',' ')
train_data['Questions']=train_data['Questions'].str.replace('</a>',' ')
train_data['Questions']=train_data['Questions'].apply(lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x)) #removing any urls
train_data['Questions'] = train_data['Questions'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3])) #removes small length words (len<3)
train_data['Questions']=train_data['Questions'].str.replace('<a href=" ">','')

train_data['Questions']=train_data['Questions'].apply(lambda x : x.lower()) #coverting to lowercase


In [7]:
test_data['Questions']=test_data['Questions'].str.replace('</p>',' ')
test_data['Questions']=test_data['Questions'].str.replace('\n',' ')
test_data['Questions']=test_data['Questions'].str.replace('</a>',' ')
test_data['Questions'] = test_data['Questions'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
test_data['Questions']=test_data['Questions'].str.replace('<a href=" ">','')
test_data['Questions']=test_data['Questions'].apply(lambda x : x.lower())
test_data['Questions']=test_data['Questions'].apply(lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x)) #removing any urls

In [8]:
#removing the stop words and the punctuations from test and train dataset


stop_words=set(nltk.corpus.stopwords.words('english'))
train_data['Questions']=train_data['Questions'].apply(lambda x:' '.join([w for w in x.split() if w not in stop_words]))
test_data['Questions']=test_data['Questions'].apply(lambda x:' '.join([w for w in x.split() if w not in stop_words]))

In [9]:
#removing the punctuations from test and train dataset


punctuations = string.punctuation
train_data['Questions']=train_data['Questions'].apply(lambda x:' '.join([''.join([char for char in w if char not in punctuations]) for w in x.split()]))
test_data['Questions']=test_data['Questions'].apply(lambda x:' '.join([''.join([char for char in w if char not in punctuations]) for w in x.split()]))



In [10]:
test_data=test_data.reset_index()


In [11]:

test_data.drop(['index'],inplace=True,axis=1)

In [12]:
#dropping the rows with empty values of Question after filtering
for j in range(len(test_data['Questions'])):
  if len(test_data['Questions'][j])==0:
     test_data.drop(j,inplace=True)

for j in range(len(train_data['Questions'])):
  if len(train_data['Questions'][j])==0:
     train_data.drop(j,inplace=True)
     print("yes")

yes
yes
yes
yes
yes


In [13]:
test_data=test_data.reset_index()
train_data=train_data.reset_index()

In [14]:
test_data.drop(['index'],inplace=True,axis=1)
train_data.drop(['index'],inplace=True,axis=1)


In [15]:
#using lemmatization on the questions of the train and test dataset
def lemmatization(text):
    pos_dict = {
        'N': 'n',  # Noun
        'V': 'v',  # Verb
        'R': 'r',  # Adverb
        'J': 'a'   # Adjective
    }
    pos_tags = pos_tag(text)
    lemmatizer = WordNetLemmatizer()
    lemma=[]
    for word, tag in pos_tags:
        if (tag[0].upper() not in pos_dict.keys()):
          pos='n'
        else:
          pos= pos_dict[tag[0].upper()]
        lemma.append(lemmatizer.lemmatize(word,pos=pos))
    return lemma

train_data['Questions']=train_data['Questions'].apply(lambda x : lemmatization(x.split()))
test_data['Questions']=test_data['Questions'].apply(lambda x : lemmatization(x.split()))
train_data["Questions"]=train_data["Questions"].apply(lambda x : " ".join(x))

test_data["Questions"]=test_data["Questions"].apply(lambda x : " ".join(x))


**Data Exploration with dev of Base Model**

In [16]:
#Analysing certain parameters about the train data

def tokenize_question(text):
    return text.split()

def tokenize_tags(text):
    return text.split('|')

questions = train_data['Questions'].tolist()
tags = train_data['Tags'].tolist()

print('The total number of words in the data is: ', sum([len(text.split()) for text in questions]))



question_vect = CountVectorizer(tokenizer=tokenize_question)
questions=question_vect.fit_transform(questions)

print('The number of words in the vocabulary is: ', len(question_vect.vocabulary_))


# Finding the most common tags

max_tags = 100
tags_vect = CountVectorizer(tokenizer=tokenize_tags, max_features=max_tags)
tags = tags_vect.fit_transform(tags)
tags = tags.toarray()
print('Number of tags: ', len(tags_vect.vocabulary_))

tags_token = tags_vect.get_feature_names_out()
tag_frequency = tags.sum(axis=0)
print('The list of tags with frequency is: ')
print(dict(zip(tags_token, tag_frequency)))



The total number of words in the data is:  2195067
The number of words in the vocabulary is:  268567
Number of tags:  100
The list of tags with frequency is: 
{'.htaccess': 29, '.net': 152, 'actionscript-3': 39, 'ajax': 42, 'algorithm': 65, 'amazon-web-services': 125, 'android': 1405, 'angular': 106, 'angularjs': 163, 'apache': 52, 'apache-spark': 36, 'arrays': 50, 'asp.net': 215, 'asp.net-mvc': 99, 'asp.net-mvc-3': 26, 'assembly': 29, 'azure': 102, 'bash': 87, 'batch-file': 31, 'c': 424, 'c#': 2028, 'c++': 958, 'cakephp': 28, 'css': 265, 'database': 90, 'delphi': 56, 'django': 151, 'docker': 98, 'eclipse': 64, 'elasticsearch': 36, 'excel': 205, 'facebook': 56, 'firebase': 37, 'flutter': 207, 'git': 153, 'go': 72, 'google-app-engine': 28, 'google-apps-script': 38, 'hadoop': 42, 'haskell': 36, 'hibernate': 28, 'html': 485, 'html5': 39, 'ios': 692, 'iphone': 279, 'java': 2324, 'javascript': 2897, 'jquery': 529, 'json': 82, 'kubernetes': 51, 'laravel': 57, 'linux': 192, 'machine-learning'

In [17]:
#checking the top most tags based on the frequency

tag_count = [[x[0],x[1]] for x in zip(tags_token, tag_frequency)]
tag_count = sorted(tag_count, key = lambda x: x[1], reverse=True)
most_common_tags=[tag[0] for tag in tag_count]
print('Rank     Tag      Count')
for i in range(100):
    print("%d   %s   %d" % (i+1, tag_count[i][0], tag_count[i][1]))

Rank     Tag      Count
1   javascript   2897
2   java   2324
3   python   2267
4   c#   2028
5   php   1712
6   android   1405
7   c++   958
8   ios   692
9   jquery   529
10   html   485
11   r   474
12   sql   472
13   c   424
14   ruby-on-rails   383
15   mysql   363
16   node.js   350
17   reactjs   300
18   iphone   279
19   css   265
20   asp.net   215
21   flutter   207
22   excel   205
23   linux   192
24   objective-c   171
25   angularjs   163
26   git   153
27   .net   152
28   django   151
29   windows   145
30   ruby   143
31   sql-server   139
32   amazon-web-services   125
33   swift   118
34   typescript   112
35   angular   106
36   azure   102
37   vb.net   101
38   python-3.x   100
39   asp.net-mvc   99
40   docker   98
41   xml   97
42   regex   96
43   wordpress   93
44   matlab   91
45   database   90
46   bash   87
47   json   82
48   postgresql   82
49   scala   79
50   spring   79
51   wpf   79
52   mongodb   74
53   powershell   74
54   go   72
55   algorithm

In [20]:
#function to calculate the evaluation metrics

def eval_metrics(y_test, y_predicted, print_metrics=True):

    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')

    if print_metrics:
        print("f1: %.3f - precision: %.3f - recall: %.3f - accuracy: %.3f" % (
            f1, precision, recall, accuracy))
    return f1, precision, recall, accuracy

In [21]:
train_data.head()

train_new=train_data.copy()
test_new=test_data.copy()

In [22]:


def tokenize_question(text):
    return text.split()

def filter_number_features(name):
  if name[0] in '0123456789' or len(name)<=3:
    return False
  return True
tfidf_vect = TfidfVectorizer(tokenizer=tokenize_question,
                               stop_words='english',
                               min_df=4,
                               max_df=0.5,max_features=7000)

X_train_tfidf = tfidf_vect.fit_transform(train_new["Questions"]).todense()
X_test_tfidf = tfidf_vect.transform(test_new["Questions"]).todense()
# print('The number of words in the vocabulary is: ', len(tfidf_vect.vocabulary_))



#get the feature names
feature_names=tfidf_vect.get_feature_names_out()

# Get the IDF scores
idf_scores = tfidf_vect.idf_

Final_Feature_Set=[]
for idx,feature_name in enumerate(feature_names):
    if filter_number_features(feature_name):
       Final_Feature_Set.append([feature_name,idf_scores[idx]])

Final_Feature_Set=sorted(Final_Feature_Set,key=lambda x :x[1],reverse=True)
Final_Feature_Set=[x[0] for x in Final_Feature_Set]


#Another approach (tfidf vectorization as feature set)

df_train = pd.DataFrame(X_train_tfidf, columns=tfidf_vect.get_feature_names_out())
df_test=pd.DataFrame(X_test_tfidf, columns=tfidf_vect.get_feature_names_out())
df_train=df_train[Final_Feature_Set]
df_test=df_test[Final_Feature_Set]

df_train["Tags"]=train_new["Tags"]
df_test["Tags"]=test_new["Tags"]


In [22]:
def tokenize_question(text):
      return text.split()

def CountVectorizer_Custom(data,Features):

  questions = data['Questions'].tolist()
  tags = data['Tags'].tolist()

  question_vect = CountVectorizer(tokenizer=tokenize_question,binary=True,vocabulary=Features)
  questions=question_vect.fit_transform(questions)

  df_train = pd.DataFrame(questions.toarray(), columns=question_vect.get_feature_names_out())
  df_train["Tags"]=tags
  return df_train

#using count vectorizer as feature set

train_new=CountVectorizer_Custom(train_new,Final_Feature_Set)
test_new=CountVectorizer_Custom(test_new,Final_Feature_Set)









In [23]:
def filter_data_by_most_common_tags(data, common_tags):
    filtered_data = data[data["Tags"].isin(common_tags)]
    return filtered_data


def one_hot(column,data): #count vectorizer feature set
  # Perform one-hot encoding using get_dummies()
  one_hot_encoded = pd.get_dummies(data[column],prefix="tag")

  # Concatenate the one-hot encoded columns with the original dataframe
  data_extended = pd.concat([data, one_hot_encoded], axis=1)
  data_extended.drop(['Tags'],inplace=True,axis=1)

  return data_extended

def label_encoding(data, most_common_tags):#tfidf vectorizer
    v = {}
    for j in range(len(most_common_tags)):
        v[most_common_tags[j]] = j
    data["Tags"] = data["Tags"].apply(lambda x: v[x] if x in v else -1)
    return data


Final_train=filter_data_by_most_common_tags(df_train,most_common_tags)
Final_test=filter_data_by_most_common_tags(df_test,most_common_tags)

#dealing with missing tags (count vectorizer)

# missing_tags = set(Final_train["Tags"]) - set(Final_test["Tags"])

# Final_train=one_hot("Tags",Final_train)

# Final_test=one_hot("Tags",Final_test)

# for tag in missing_tags:
#     Final_test["tag_"+tag] = 0

# Final_test=Final_test[Final_train.columns] # in order to maintain order of the columns in test and train



#Another Approach (tfidf vectorizer)
Final_train=label_encoding(Final_train,most_common_tags)
Final_test=label_encoding(Final_test,most_common_tags)



In [26]:
X_train=Final_train.iloc[:,:-1]

Y_train=Final_train.iloc[:,-1]
X_test=Final_test.iloc[:,:-1]

Y_test=Final_test.iloc[:,-1]

In [26]:
%%time
log_clf = OneVsRestClassifier(LogisticRegression())

log_clf.fit(X_train, Y_train)



OneVsRestClassifier(estimator=LogisticRegression())

In [27]:
Y_train_predict = log_clf.predict(X_train)
eval_metrics(Y_train, Y_train_predict)

f1: 0.818 - precision: 0.952 - recall: 0.721 - accuracy: 0.700


(0.8182297686727024, 0.9519304302097099, 0.7212010549807263, 0.699655102454859)

In [28]:
Y_test_predict=log_clf.predict(X_test)
eval_metrics(Y_test,Y_test_predict)

f1: 0.383 - precision: 0.639 - recall: 0.290 - accuracy: 0.263


(0.38286240616119727,
 0.6393702947555568,
 0.29014241831890536,
 0.2626361351577772)

In [ ]:
# Classifier - Algorithm - SVM

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train,Y_train)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Y_test)*100)

In [33]:
# NB classifier
Naive = naive_bayes.MultinomialNB()

Naive.fit(X_train,Y_train)
# predict the labels on validation dataset

predictions_NB = Naive.predict(X_test)

# Use accuracy_score function to get the accuracy

print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Y_test)*100)

Naive Bayes Accuracy Score ->  33.44472291840713
